In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date, datetime

In [3]:
    life_table_dict = {}
    life_table_dict[2016] = {}
    life_table_dict[2017] = {}
    life_table_dict[2018] = {}
    life_table_dict[2016][0] = pd.read_excel("../../life_tables/2016/Males/life_table_2016_male.xlsx").to_numpy()[2:-1,1]
    life_table_dict[2017][0] = pd.read_excel("../../life_tables/2017/Male/life_table_2017_male.xlsx").to_numpy()[2:-1,1]
    life_table_dict[2018][0] = pd.read_excel("../../life_tables/2018/Males/life_table_2018_male.xlsx").to_numpy()[2:-1,1]
    life_table_dict[2016][1] = pd.read_excel("../../life_tables/2016/Females/life_table_2016_female.xlsx").to_numpy()[2:-1,1]
    life_table_dict[2017][1] = pd.read_excel("../../life_tables/2017/Female/life_table_2017_female.xlsx").to_numpy()[2:-1,1]
    life_table_dict[2018][1] = pd.read_excel("../../life_tables/2018/Females/life_table_2018_female.xlsx").to_numpy()[2:-1,1]

    pop_file_name = "../../path_data/age_individual_October21_renamed.xlsx"
    pop_df = pd.read_excel(pop_file_name)

    beta234_f = "../../beta_estimates/beta_estimates_234.xlsx"
    beta15_f = "../../beta_estimates/beta_estimates_15.xlsx"
    beta234_arr = pd.read_excel(beta234_f).to_numpy()[:,2:]
    beta15_arr = pd.read_excel(beta15_f).to_numpy()[:,2:]

In [5]:
        pop_arr = pop_df.to_numpy(dtype=np.float64)

        arr234 = np.asarray([row for row in pop_arr 
                  if (row[4] == 2 or row[4] == 3 or row[4] == 4
                  or row[3] == 2 or row[3] == 3 or row[3] == 4)], dtype=np.float64)

In [7]:
def path_to_logistic_form(a):
    s2 = a[:,3]
    s3 = a[:,4]
    ia = a[:,5]
    a = np.concatenate([
        np.ones((len(a), 1)),
        (s2 == 1)[:,np.newaxis],
        (s2 == 2)[:,np.newaxis],
        (s2 == 3)[:,np.newaxis],
        (s2 == 4)[:,np.newaxis],
        (s3 == 1)[:,np.newaxis],
        (s3 == 2)[:,np.newaxis],
        (s3 == 3)[:,np.newaxis],
        (s3 == 4)[:,np.newaxis],
        (ia == 1)[:,np.newaxis],
        (ia == 2)[:,np.newaxis],
        a[:,2][:,np.newaxis],
        a[:,9][:,np.newaxis],
        a[:,1][:,np.newaxis] - 1, # change sex from {1,2} to {0,1}
        a[:,6][:,np.newaxis] - 1, # change poverty from {1,2} to {0,1}
        a[:,10][:,np.newaxis],
        a[:,8][:,np.newaxis],
        a[:,0][:,np.newaxis],
        np.zeros((a.shape[0],1)),
    ], axis=1, dtype=np.float64)
    return a

In [6]:
arr234.shape

(9533, 11)

In [12]:
l = []

# 0 agegrp
# 1 sex
# 2 black
# 3 state2
# 4 state3
# 5 ia
# 6 pov
# 7 set
# 8 weight
# 9 age
# 10 start_age

for s2 in range(1,5):
    for s3 in range(1,5):
        for ia in range(1,3):
            for black in range(2):
                for sex in range(1,3):
                    for pov in range(1,3):
                        l.append([
                            0,
                            sex,
                            black, 
                            s2, 
                            s3, 
                            ia,
                            pov, 
                            0, 
                            0, 
                            30, 
                            20, 
                        ])

In [13]:
len(l)

256

In [14]:
l[0]

[0, 1, 0, 1, 1, 1, 1, 0, 0, 30, 20]

In [15]:
arr_pop = np.asarray(l)
arr234 = np.asarray([row for row in arr_pop 
         if (row[4] == 2 or row[4] == 3 or row[4] == 4
         or row[3] == 2 or row[3] == 3 or row[3] == 4)], dtype=np.float64)

In [16]:
arr234_aug = path_to_logistic_form(arr234)
arr234_aug[:,-1] = np.ones((arr234.shape[0]))


In [18]:
beta234_arr = beta234_arr.astype(np.float64)
beta_234_aug = np.concatenate([
    beta234_arr[:,:5],
    np.zeros((len(beta234_arr), 1)),
    beta234_arr[:,5:],
    np.zeros((len(beta234_arr), 4)),
], axis=1, dtype=np.float64)

beta_234_aug = np.transpose(beta_234_aug)

In [19]:
logits_234 = np.matmul(arr234_aug, beta_234_aug).astype(np.float64)

In [20]:
exps = np.exp(logits_234)
p4 = 1 / (1 + np.sum(exps, axis=1))
probs234 = np.asarray([
    p4*exps[:,0], # p2
    p4*exps[:,1], # p3
    p4,           # p4
    p4*exps[:,2], # p5
], dtype=np.float64).transpose()

In [22]:
probs234.shape

(240, 4)

In [23]:
arr234.shape

(240, 11)

In [24]:
together_pop_probs = np.concatenate([arr234, probs234], axis=1)

In [28]:
l = list(pop_df.columns)
l.append("prob 2")
l.append("prob 3")
l.append("prob 4")
l.append("prob 5")
l

['agegrp',
 'sex',
 'black',
 'state_2',
 'state_3',
 'initiation_age_grp',
 'poverty',
 'set',
 'weight',
 'age',
 'start_age',
 'prob 2',
 'prob 3',
 'prob 4',
 'prob 5']

In [29]:
together_df = pd.DataFrame(data=together_pop_probs, index=None, columns=l)
together_df

,agegrp,sex,black,state_2,state_3,initiation_age_grp,poverty,set,weight,age,start_age,prob 2,prob 3,prob 4,prob 5
0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,30.0,20.0,0.720388,0.118384,0.121962,0.039266
1,0.0,1.0,0.0,1.0,2.0,1.0,2.0,0.0,0.0,30.0,20.0,0.801896,0.078953,0.089858,0.029293
2,0.0,2.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,30.0,20.0,0.716127,0.125043,0.117574,0.041256
3,0.0,2.0,0.0,1.0,2.0,1.0,2.0,0.0,0.0,30.0,20.0,0.798791,0.083565,0.086803,0.030841
4,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,30.0,20.0,0.702712,0.194781,0.076519,0.025988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.0,2.0,0.0,4.0,4.0,2.0,2.0,0.0,0.0,30.0,20.0,0.056629,0.025844,0.877457,0.040069
236,0.0,1.0,1.0,4.0,4.0,2.0,1.0,0.0,0.0,30.0,20.0,0.054308,0.065670,0.843215,0.036807
237,0.0,1.0,1.0,4.0,4.0,2.0,2.0,0.0,0.0,30.0,20.0,0.080286,0.058165,0.825082,0.036467
238,0.0,2.0,1.0,4.0,4.0,2.0,1.0,0.0,0.0,30.0,20.0,0.055377,0.071150,0.833805,0.039669


In [30]:
from IPython.display import display, HTML

In [31]:
display(together_df)

,agegrp,sex,black,state_2,state_3,initiation_age_grp,poverty,set,weight,age,start_age,prob 2,prob 3,prob 4,prob 5
0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,30.0,20.0,0.720388,0.118384,0.121962,0.039266
1,0.0,1.0,0.0,1.0,2.0,1.0,2.0,0.0,0.0,30.0,20.0,0.801896,0.078953,0.089858,0.029293
2,0.0,2.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,30.0,20.0,0.716127,0.125043,0.117574,0.041256
3,0.0,2.0,0.0,1.0,2.0,1.0,2.0,0.0,0.0,30.0,20.0,0.798791,0.083565,0.086803,0.030841
4,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,30.0,20.0,0.702712,0.194781,0.076519,0.025988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.0,2.0,0.0,4.0,4.0,2.0,2.0,0.0,0.0,30.0,20.0,0.056629,0.025844,0.877457,0.040069
236,0.0,1.0,1.0,4.0,4.0,2.0,1.0,0.0,0.0,30.0,20.0,0.054308,0.065670,0.843215,0.036807
237,0.0,1.0,1.0,4.0,4.0,2.0,2.0,0.0,0.0,30.0,20.0,0.080286,0.058165,0.825082,0.036467
238,0.0,2.0,1.0,4.0,4.0,2.0,1.0,0.0,0.0,30.0,20.0,0.055377,0.071150,0.833805,0.039669


In [32]:
together_df.to_excel("test_outputs.xlsx")